In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
from solver_output import run_solver_shift_return


/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Copied and Pasted from Alberto's readme


The solver is a single Python script, contained in `solver/solver.py`.
It takes the following parameters:
* `-m` to specify the model name (`base`, `fixed`, `partflex`, or `flex`).
* `-i` to specify the location of the instance file.
* `-c` to specify the outsourcing cost multiplier (parameter OC in the paper).
* `-r` to specify the regional bound multiplier (parameter RM in the paper).
* `-g` to specify the global bound multiplier (parameter GM in the paper).
* `-u` to specify the maximum number of shift start times for the `partflex` model (parameter $\mu$ in the paper).
* `-o` to specify the location of the JSON solution file produced by the solver.

In [8]:
#Export the raw schduling problem solutions

# # #stopped running cause it would be a gazillion things and we can run it if/when we need it
# # # for file in os.listdir(r'../instances'):
# # for file in ['berlin_db=0.50_dt=atend.json','berlin_db=0.50_dt=doublepeak.json','berlin_db=0.50_dt=peak.json','berlin_db=0.50_dt=peak.json']:
# #     instance_ = f"../instances/{file}"
# #     for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
# #         for RM in [0.75, 1.0, 1.5, 3.0, 5.0]:
# #             for GM in [0.6, 0.7, 0.8, 0.9, 1.0]:
# #                 run_solver_output(model='fixed', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)
# #                 run_solver_output(model='flex', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)    
# #                 for max_n_shift in range(2,5):
# #                     run_solver_output(model='partflex', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)

# #stopped running cause it would be a gazillion things and we can run it if/when we need it
# # for file in os.listdir(r'../instances'):
# for file in ['berlin_db=0.50_dt=atend.json','berlin_db=0.50_dt=doublepeak.json','berlin_db=0.50_dt=peak.json','berlin_db=0.50_dt=peak.json']:
#     instance_ = f"../instances/{file}"
#     for OC in [1.2]:
#         for RM in [0.75]:
#             for GM in [0.6]:
#                 run_solver_output(model='fixed', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)
#                 run_solver_output(model='flex', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)    
#                 for max_n_shift in range(2,5):
#                     run_solver_output(model='partflex', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)


In [9]:
#Load and concat example raw scheduling solutions

# list_output = []

# for filename in os.listdir(r'../results/results_output'):
#     with open(f'../results/results_output/{filename}', 'r') as file:
#         data = json.load(file)
#         df_ = pd.DataFrame(data)
#         list_output.append(df_)

# df_output = pd.concat(list_output, ignore_index = True)
# df_output.to_excel(r'../results/df_raw_output_example.xlsx', index = False)


In [3]:
#Code that creates JSON file that outputs optimal shifts from scheduling solution

# dummy = 0
# dict_raw_out = {}

for file in os.listdir(r'../instances'):
    if (file.find('doublepeak') > -1) or (file.find('uniform') > -1):
        if file not in os.listdir(r'../shifts'):
            shift_out = f"../shifts/{file}"
            dict_out = {
                'instance_file':[],
                'city':[],
                'demand_baseline':[],
                'demand_type':[],
                'model':[],
                'max_n_shifts':[],
                'outsourcing_cost_multiplier':[],
                'regional_multiplier':[],
                'global_multiplier':[],
                'region':[],
                'shifts_start':[],
                'shifts_end':[]
            }

            city_pattern = r'(\w+)_db'
            db_pattern = r'db=(\d+\.\d+)'
            dt_pattern = r'dt=(\w+)'

            city_match = re.search(city_pattern, file)
            db_match = re.search(db_pattern, file)
            dt_match = re.search(dt_pattern, file)
            
            city = city_match.group(1) if city_match else None
            demand_baseline = float(db_match.group(1)) if db_match else None
            demand_type = dt_match.group(1) if dt_match else None

            instance_ = f"../instances/{file}"

            for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
                for RM in [1.5]:
                    for GM in [0.8]:
                        for model in ['fixed','flex','partflex']:
                            if model == 'partflex':
                                for max_n_shift in range(2,5):
                                    dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)
                                    # if max_n_shift == 2:
                                    #     dict_raw_out = dict_raw.copy()
                                    dict_base = {
                                        'instance_file':[file]*n_regions,
                                        'city':[city]*n_regions,
                                        'demand_baseline':[demand_baseline]*n_regions,
                                        'demand_type':[demand_type]*n_regions,
                                        'model':[model]*n_regions,
                                        'max_n_shifts':[max_n_shift]*n_regions,
                                        'outsourcing_cost_multiplier':[OC]*n_regions,
                                        'regional_multiplier':[RM]*n_regions,
                                        'global_multiplier':[GM]*n_regions,
                                        'region':[region for region in range(0,n_regions)]
                                    }
                                    list_shift_start = []
                                    list_shift_end = []
                                    for region in range(0, n_regions):
                                        if region in dict_shifts.keys():
                                            list_shift_start.append(dict_shifts[region]['shifts_start'])
                                            list_shift_end.append(dict_shifts[region]['shifts_end'])
                                        else:
                                            list_shift_start.append({})
                                            list_shift_end.append({})
                                    dict_base['shifts_start'] = list_shift_start
                                    dict_base['shifts_end'] = list_shift_end
                                    for key in dict_base.keys():
                                        dict_out[key].extend(dict_base[key])
                            else:
                                dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)
                                dict_base = {
                                    'instance_file':[file]*n_regions,
                                    'city':[city]*n_regions,
                                    'demand_baseline':[demand_baseline]*n_regions,
                                    'demand_type':[demand_type]*n_regions,
                                    'model':[model]*n_regions,
                                    'max_n_shifts':[np.nan]*n_regions,
                                    'outsourcing_cost_multiplier':[OC]*n_regions,
                                    'regional_multiplier':[RM]*n_regions,
                                    'global_multiplier':[GM]*n_regions,
                                    'region':[region for region in range(0,n_regions)]
                                }
                                list_shift_start = []
                                list_shift_end = []
                                for region in range(0, n_regions):
                                    if region in dict_shifts.keys():
                                        list_shift_start.append(dict_shifts[region]['shifts_start'])
                                        list_shift_end.append(dict_shifts[region]['shifts_end'])
                                    else:
                                        list_shift_start.append({})
                                        list_shift_end.append({})
                                dict_base['shifts_start'] = list_shift_start
                                dict_base['shifts_end'] = list_shift_end
                                for key in dict_base.keys():
                                    dict_out[key].extend(dict_base[key])

            with open(shift_out, 'w') as f:
                json.dump(dict_out, f, indent=2)


in instance
in solver
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14578 rows, 21784 columns and 32392 nonzeros
Model fingerprint: 0x5f4eb271
Variable types: 14160 continuous, 7624 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 1107.4400054
Found heuristic solution: objective 1107.4399922
Presolve removed 14135 rows and 18358 columns
Presolve time: 0.09s
Presolved: 443 rows, 3426 columns, 7223 nonzeros
Found heuristic solution: objective 853.3666767
Variable types: 462 continuous, 2964 integer (45 binary)

Root re

In [9]:
#this is an example output of the shift creation code

# list_output = []

# for filename in os.listdir(r'../shifts'):
#     if filename == 'berlin_db=0.50_dt=atend.json':
#         with open(f'../shifts/{filename}', 'r') as file:
#             data = json.load(file)
#             df_ = pd.DataFrame(data)
#             list_output.append(df_)

# df_output = pd.concat(list_output, ignore_index = True)
# df_output.to_excel(r'../results/df_shifts_example.xlsx', index = False)



In [35]:
#Function code to load the shift information

def load_shift(level, instance_file, OC, RM, GM, model, max_n_shifts):
    with open(f'../shifts/{instance_file}', 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]

    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start','shifts_end']]

    dict_shifts = df_shifts.set_index('region').to_dict(orient='index')

    if level == 'region':
        #return dictionary of dictionaries
        #each key is a region, first element is dictionary of shift_start, second element is shift_end
        return dict_shifts        
    else:
        list_start = []
        for region in dict_shifts.keys():
            for shift_index in dict_shifts[region]['shifts_start'].keys():
                list_start.append(dict_shifts[region]['shifts_start'][shift_index])
        list_start = list(set(list_start))

        dict_return = {}
        dict_return['shifts_start'] = {}
        dict_return['shifts_end'] = {}

        for shift_index in range(0,len(list_start)):
            dict_return['shifts_start'][shift_index] = list_start[shift_index]
            dict_return['shifts_end'][shift_index] = list_start[shift_index] + 4
        return dict_return


In [37]:
#How to load shift information

#variables that should alread be established
level = 'region' #this can be shifts on the region level or global level
instance_file = 'berlin_db=0.50_dt=atend.json'
OC = 1.2
RM = 0.75
GM = 0.6
model = 'partflex' #can be ('fixed','flex','partflex')
max_n_shifts = 2 #when model = 'partflex' can be 2, 3, or 4


dict_ = load_shift('region', instance_file, OC, RM, GM, model, max_n_shifts)

print(f'regional level shift: {dict_}')

dict_ = load_shift('global', instance_file, OC, RM, GM, model, max_n_shifts)

print(f'global level shift: {dict_}')


regional level shift: {0: {'shifts_start': {'0': 3}, 'shifts_end': {'0': 7}}, 1: {'shifts_start': {'0': 3}, 'shifts_end': {'0': 7}}, 2: {'shifts_start': {'0': 3, '1': 4}, 'shifts_end': {'0': 7, '1': 8}}, 3: {'shifts_start': {}, 'shifts_end': {}}}
global level shift: {'shifts_start': {0: 3, 1: 4}, 'shifts_end': {0: 7, 1: 8}}
